In [9]:
import torch
import torch.nn as nn

class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        # embed size가 256, heads가 8이라면 8*32로 나누어질 수 있다 -> 어떻게 나누어지는가?
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size            # embed_size = 256
        self.heads = heads                      # heads = 8
        self.head_dim = embed_size // heads     # head_dim = 32
                                                # 당연히 head_dim은 integer이므로 assert로 확인한다!
            
        assert (self.head_dim * heads == embed_size), 'Embed size needs to be div by heads'
        
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys   = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads*self.head_dim, embed_size)

        """
        * 차원을 head_dim(32)로 나누는 이유는 결국 나중에 Multihead attention으로 전환하기 위해서이다.
        * query / value / key는 모두 같은 차원
        * query = attention의 수혜자 (단어 하나)
        * keys = attention의 대상 (전체 단어 / 어디에 집중할 것이여!)
        * values = softmax 값을 얹는 대상
        * fc_out = heads들을 모두 concat하여 fc_out에 집어넣는다!
        """
        
    
    def forward(self, values, keys, query, mask):
        N = query.shape[0]    # How many inputs we are going to send at the same time
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
        # These Three will be correspond to source / target sentence length
        # 단순히 차원수이며, src / trg에 비례하는 이유는 heads 갯수에 따라 달라질 거기 때문에
        
        
        # Split embedding into self.heads pieces
        
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, key_len, self.heads, self.head_dim)

        """
        query는 values / keys와 다르게 복수형이 아니다 -> 즉, 단일한 값이라는 거다.
        
        """
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # n: batch_size
        # q: query_len
        # h: heads
        # d: heads dimension
        # k: key_len
        # einsum: bmm을 좀 더 쉽게 할 수 있따.
        # 차원을 알파벳으로 구성한 후 내가 원하는 차원의 Form으로 BMM으로 구성
        
        """
        matrix multiplication for various dimensions
        queries shape: (N, query_len, heads, heads_dim)
        keys shape: (N, key_len, heads, head_dim)
        energy shape: (N, heads, query_len, key_len)
        --> query_len: target / src sentence , key_len: src sentence
        query len이 얼만큼 key_len에 집중할 것인가?
        """
        if mask is not Noe:
            energy = energy.masked_fill(mask == 0, float("-1e20"))
            
        attention = torch.softmax(energy / (self.embed_size ** (1/2)), dim=3)
        # now multiply with value
        
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads*self.head_dim
        )
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # out return value = (N, query_len, heads, head_dim)
        
        # after einsum (N, query_len, heads, head_dim) then flatten last two dimensions
        
        out = self.fc_out(out)
        return out
    
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        
        # Layernorm - BatchNorm 은 매우 유사
        # BatchNorm은 배치의 평균을 구한 뒤 Normalize
        # Layer는 각 예시의 평균을 구한 뒤 Normalize
        
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            # forward_expansion??
            nn.ReLU(),
            nn.Linear(forward_expansion*embed_size, embed_size)
        )
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)
        
        add_norm = self.norm1(attention + query)
        x = self.dropout(add_norm)
        
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

        # attention + query 자체가 skip connection
        # skip connection => 입력과 출력을 다음 레이어의 입력으로 보내는 것을 의미함
        
class Encoder(nn.Module):
    def __init(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length
        # related to positional embedding
        # positional embedding은 문장 길이에 영향을 받으므로, 문장의 최대 길이를 input으로 넣어줘야 한다.
    ):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.positional_embedding = nn.Embedding(max_length, embed_size)
        
        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                embed_size,
                heads,
                dropout = dropout,
                forward_expansion = forward_expansion)
            ]
        )
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        # range(a, b) = a부터 b까지
        # arange(a, b) = a부터 b-1까지
        # 왜 positional embedding에서 sinusidal function을 사용하지 않는 것이요?
        
        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))
        
        for layer in self.layers:
            out = layer(out, out, out, mask)
            
        return out
    
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        
        self.attention = SelfAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, value, key, src_mask, trg_mask):
        # trg_mask는 필수이지만 src_mask는 선택이다.
        # src_mask는 <pad>에 대하여 masking을 하여 계산을 하지 않기 위해서이기 때문이다.
        
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out
            
class Decoder(nn.Module):
    def __init__(self,
                trg_vocab_size,
                embed_size,
                num_layers,
                heads,
                forward_expansion,
                dropout, device, max_length):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)
        self.layers = nn.ModuleList(
            [DecoderBlock(embed_size, heads, forward_expansion, dropout, device) 
             for _ in range(num_layers)]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x) + slef.position_embedding(positions)))
        
        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)
        
        out = self.fc_out(x)
        
        return out
    
class Transformer(nn.Module):
    def __init__(self,
                 src_vocab_size,
                 trg_vocab_size,
                 src_pad_idx,
                 trg_pad_idx,
                 embed_size=256,
                 num_layers = 6,
                 forward_expansion = 4,
                 heads=8,
                 dropout =0,
                 device='cuda',
                 max_length=100):
        super(Transformer, self).__init__()
        
        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length)
        
        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers, 
            heads,
            forward_expansion,
            dropout,
            device,
            max_length)
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsequeeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)
    
    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )
        return trg_mask.to(self.device)
    
        # tril => triangular lower
            
    def forawrd(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out
            
                 

In [1]:
import torch

In [6]:
torch.arange(0, 30).expand(5, 30)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

In [8]:
torch.range(0, 30)

/usr/local/var/pyenv/versions/torch_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  """Entry point for launching an IPython kernel.


tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27.,
        28., 29., 30.])